In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home1/zzhang201@kgi.edu/GAN/zzGAN/')

In [3]:
import tensorflow as tf
# import tensorflow_gan as tfgan

2025-09-16 17:13:26.081816: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-16 17:13:26.116037: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
print("TF version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))

TF version: 2.16.1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [5]:
import matplotlib.pyplot as plt
import io
from PIL import Image
import numpy as np
from scipy.spatial.distance import jensenshannon
import matplotlib.ticker as ticker

In [6]:
from sklearn.manifold import TSNE
import umap

/home1/zzhang201@kgi.edu/.conda/envs/tf2_16/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os
import json
import random
import datetime
import platform

In [8]:
import importlib
import gan.documentation

In [9]:
from gan.models import get_model, get_specific_hooks
from gan.parameters import get_flags
from gan.documentation import setup_logdir, get_properties
from gan.documentation import print_run_meta_data, add_model_metadata
from tensorflow.keras import mixed_precision
from tensorflow.keras.mixed_precision import LossScaleOptimizer
# Enable global mixed precision policy
mixed_precision.set_global_policy('mixed_float16')
from protein.quality_gates import quality_losses, anarci_quality_log

In [10]:
class FakeFlags:
    # General model setup
    model_type = 'wgan'
    architecture = 'gumbel'
    batch_size = 64
    z_dim = 128
    gf_dim = 64
    df_dim = 64
    dim = gf_dim
    attn_pos = 2

    # Kernel and dilation config
    kernel_height = 3
    kernel_width = 3
    dilation_rate = 2
    pooling = 'conv'

    # Optional: logging / save frequency
    name = 'multiG_attn_adjusted_embed'
    steps = 200000
    save_summary_steps = 1000
    save_checkpoint_sec = 5000

    # Optimizer settings
    generator_learning_rate = 1e-4
    discriminator_learning_rate = 1e-4
    beta1 = 0.5
    beta2 = 0.9

    # Dataset & file structure
    dataset = 'zz'
    seq_length = 160
    logdir = '/home1/zzhang201@kgi.edu/GAN/zzGAN/logs/zz'
    # properties_file = 'properties.json'

    # Misc
    seed = 950806
    is_train = True
    multid_schedule = 20000
    d_step = 3
    shuffle_buffer_size = 1000
    label_noise_level = 0.0
    noise_level = 0.0

    def flag_values_dict(self):
        return {k: getattr(self, k) for k in dir(self)
                if not k.startswith("__") and not callable(getattr(self, k))}
FLAGS = FakeFlags()

In [ ]:
random.seed(FLAGS.seed)
np.random.seed(FLAGS.seed)
tf.random.set_seed(FLAGS.seed)

In [11]:
def save_flags(flags, log_dir, filename="flags.json"):
    """Save FLAGS to a JSON file in the log directory."""
    os.makedirs(log_dir, exist_ok=True)
    file_path = os.path.join(log_dir, filename)

    with open(file_path, "w") as f:
        json.dump(flags.flag_values_dict(), f, indent=4)
    
    print(f"[FLAGS] Saved flags to {file_path}")

def load_flags(log_dir, filename="flags.json"):
    """Load FLAGS from JSON file and return as a FakeFlags object."""
    file_path = os.path.join(log_dir, filename)

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"[FLAGS] No flags file found at {file_path}")

    with open(file_path, "r") as f:
        flag_dict = json.load(f)

    # Create a new FakeFlags object and set attributes
    loaded_flags = FakeFlags()
    for k, v in flag_dict.items():
        setattr(loaded_flags, k, v)

    print(f"[FLAGS] Loaded flags from {file_path}")
    return loaded_flags


In [12]:
def parse_example(example_proto, seq_len=160, vocab_size=21):
    feature_description = {
        "sequence": tf.io.VarLenFeature(tf.int64),
        "label": tf.io.FixedLenFeature([], tf.int64),  # parsed but unused
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)

    sequence = tf.sparse.to_dense(parsed["sequence"])
    sequence = tf.cast(sequence, tf.int32)

    # Clip or pad to fixed length
    sequence = sequence[:seq_len]
    paddings = [[0, tf.maximum(0, seq_len - tf.shape(sequence)[0])]]
    sequence = tf.pad(sequence, paddings)

    # One-hot encode → [seq_len, vocab_size] float32
    one_hot = tf.one_hot(sequence, vocab_size)
    return one_hot

def load_tfrecord_dataset(
    tfrecord_dir,
    batch_size=8,
    seq_len=160,
    vocab_size=21,
    shuffle_buffer=10000,
    seed=1337,
    deterministic=True,
    cycle_length=4,
    drop_remainder=True,
):
    # Stable file order (DO NOT shuffle here)
    files = sorted(tf.io.gfile.glob(os.path.join(tfrecord_dir, "*.tfrecords")))
    if not files:
        raise FileNotFoundError(f"No .tfrecords under {tfrecord_dir}")

    ds_files = tf.data.Dataset.from_tensor_slices(files)

    # Deterministic interleave over files
    ds = ds_files.interleave(
        lambda p: tf.data.TFRecordDataset(p),
        cycle_length=cycle_length,
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=deterministic,
    )

    ds = ds.map(
        lambda ex: parse_example(ex, seq_len, vocab_size),
        num_parallel_calls=tf.data.AUTOTUNE,
    )

    # Seeded shuffle; NO reshuffle each epoch → same order every run
    if shuffle_buffer and shuffle_buffer > 0:
        ds = ds.shuffle(
            buffer_size=shuffle_buffer,
            seed=seed,
            reshuffle_each_iteration=False,
        )

    # Infinite stream with fixed batch shapes
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)
    ds = ds.repeat()

    # Enforce deterministic behavior at the pipeline level if requested
    opts = tf.data.Options()
    opts.deterministic = deterministic
    ds = ds.with_options(opts)

    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [13]:
dataset = load_tfrecord_dataset(
    tfrecord_dir="/home1/zzhang201@kgi.edu/GAN/zzGAN/gan/data/zz/train",
    batch_size=FLAGS.batch_size,
    seq_len=160,
    seed=FLAGS.seed,
    deterministic=True,  # flip to False if you want max throughput
)

batch = next(iter(dataset))
print(batch.shape)

2025-09-16 17:13:36.696487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43622 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:21:00.0, compute capability: 8.9
2025-09-16 17:13:36.697018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43622 MB memory:  -> device: 1, name: NVIDIA L40S, pci bus id: 0000:e1:00.0, compute capability: 8.9


(64, 160, 21)


In [14]:
# properties = get_properties(FLAGS)
# properties

In [15]:
FLAGS.architecture

'gumbel'

In [16]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join(FLAGS.logdir, FLAGS.name, timestamp)

noise = tf.random.normal([FLAGS.batch_size, FLAGS.z_dim])
model = get_model(FLAGS, log_dir, noise)

In [17]:
@tf.function
def train_discriminator_step(generator, discriminator, d_optimizer, real_batch, noise_dim, lambda_gp=10.0):
    batch_size = tf.shape(real_batch)[0]
    real_batch = tf.expand_dims(real_batch, axis=1)
    noise = tf.random.normal([batch_size, noise_dim])
    fake_batch = generator(noise, training=True, return_hard=False)

    with tf.GradientTape() as disc_tape:
        real_logits = discriminator(real_batch, training=True)
        fake_logits = discriminator(fake_batch, training=True)
        d_loss = tf.reduce_mean(fake_logits) - tf.reduce_mean(real_logits)

        # Gradient penalty
        alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
        interpolated = alpha * real_batch + (1. - alpha) * fake_batch
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            interp_logits = discriminator(interpolated, training=True)
        grads = gp_tape.gradient(interp_logits, [interpolated])[0]
        grads_flat = tf.reshape(grads, [batch_size, -1])
        grad_norm = tf.norm(grads_flat, axis=1)
        gp = tf.reduce_mean((grad_norm - 1.0) ** 2)
        d_loss += lambda_gp * gp

    d_grads = disc_tape.gradient(d_loss, discriminator.trainable_variables)
    d_grad_norm = tf.linalg.global_norm(d_grads)
    d_optimizer.apply_gradients(zip(d_grads, discriminator.trainable_variables))

    return d_loss, d_grad_norm


In [18]:
@tf.function
def train_generator_step(generator, discriminator, g_optimizer, noise_dim):
    noise = tf.random.normal([FLAGS.batch_size, noise_dim])
    with tf.GradientTape() as gen_tape:
        fake_batch = generator(noise, training=True)
        fake_logits = discriminator(fake_batch, training=True)
        # After computing generator logits
        logits = model.g_model.last_logits  # You may need to store this in call()
        softmax = tf.nn.softmax(logits, axis=-1)
        entropy = -tf.reduce_mean(tf.reduce_sum(softmax * tf.math.log(softmax + 1e-8), axis=-1))
        g_loss = -tf.reduce_mean(fake_logits)
        g_loss += 0.01 * entropy  # weight of bonus (tunable)
        
    g_grads = gen_tape.gradient(g_loss, generator.trainable_variables)
    g_grad_norm = tf.linalg.global_norm(g_grads)
    g_optimizer.apply_gradients(zip(g_grads, generator.trainable_variables))

    return g_loss, g_grad_norm


In [19]:
# === Setup ===
MAX_STEPS = FLAGS.steps
NOISE_DIM = 128  # Confirm this matches your model config

checkpoint_dir = os.path.join(log_dir, "checkpoints")
summary_dir = os.path.join(log_dir, "summaries")
attn_dir = os.path.join(log_dir, "attn_scores")
# config_dir = os.path.join(FLAGS.logdir, FLAGS.name, "setup")

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(summary_dir, exist_ok=True)
os.makedirs(attn_dir, exist_ok=True)

summary_writer = tf.summary.create_file_writer(summary_dir)
save_flags(FLAGS, summary_dir)

[FLAGS] Saved flags to /home1/zzhang201@kgi.edu/GAN/zzGAN/logs/zz/multiG_attn_adjusted_embed/20250916-171337/summaries/flags.json


In [ ]:
from pathlib import Path
import csv
import re

# Where to put generated sequences for this run
SEQ_DIR = log_dir / "generated_sequences" # keep alongside your "summaries"
SEQ_DIR.mkdir(parents=True, exist_ok=True)

FASTA_PATH = SEQ_DIR / "generated_sequences.fasta"
INDEX_PATH = SEQ_DIR / "generated_sequences_index.csv"

# Write CSV header once if file is new
if not INDEX_PATH.exists():
    with open(INDEX_PATH, "w", newline="") as f:
        csv.writer(f).writerow(["step","seq_idx","length"])
        
AA = set("ACDEFGHIKLMNPQRSTVWYXBZ-")
def sanitize_seq(s: str) -> str:
    s = re.sub(r"\s+", "", s).upper()
    return "".join(ch for ch in s if ch in AA)

def append_generated_sequences(step: int, sequences):
    """Append sequences to FASTA + index CSV. Header stores the step + idx."""
    with open(FASTA_PATH, "a") as ffa, open(INDEX_PATH, "a", newline="") as fcsv:
        w = csv.writer(fcsv)
        for i, s in enumerate(sequences):
            seq = sanitize_seq(s)
            header = f"step{step:06d}_idx{i}"
            ffa.write(f">{header}\n{seq}\n")
            w.writerow([step, i, len(seq)])


In [20]:
ckpt = tf.train.Checkpoint(
    generator=model.g_model,
    discriminator=model.d_model,
    g_optimizer=model.g_optim,
    d_optimizer=model.d_optim,
    step=model.g_model.global_step
)

In [21]:
### Load old models
# ckpt.restore(os.path.join(checkpoint_dir, "ckpt-200000")).expect_partial()

In [22]:
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=3)


if ckpt_manager.latest_checkpoint:
    print(f"Restoring from {ckpt_manager.latest_checkpoint}")
    ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
else:
    print("Initializing from scratch.")

Initializing from scratch.


In [23]:
AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"  # 20 residues
AA_TO_IDX = {aa: i + 1 for i, aa in enumerate(AMINO_ACIDS)}
PAD_IDX = 0
VOCAB_SIZE = 21  # 0 (PAD) + 1–20

MAX_LEN = 160

def decode_sequence(onehot):
    tokens = tf.argmax(onehot, axis=-1).numpy().flatten()
    return ''.join([AMINO_ACIDS[i - 1] if i != 0 else '-' for i in tokens])

def decode_batch(batch_tensor: tf.Tensor) -> list[str]:
    """
    Decodes a batch of one-hot encoded tensors into amino acid sequences.

    Supports input of shape:
        - (B, 1, L, 21)
        - (B, L, 21)

    Returns:
        List of decoded strings of length B
    """
    if batch_tensor.shape.rank == 4:
        batch_tensor = tf.squeeze(batch_tensor, axis=1)  # (B, L, 21)

    return [decode_sequence(seq) for seq in batch_tensor]


def encode_sequence(seq):
    """
    Encode a sequence into one-hot with shape [1, MAX_LEN, 21]
    """
    seq = seq.upper()
    indices = [AA_TO_IDX.get(aa, PAD_IDX) for aa in seq[:MAX_LEN]]
    if len(indices) < MAX_LEN:
        indices += [PAD_IDX] * (MAX_LEN - len(indices))
    one_hot = tf.one_hot(indices, depth=VOCAB_SIZE, dtype=tf.float32)  # [MAX_LEN, 21]
    return tf.expand_dims(one_hot, axis=0)  # [1, MAX_LEN, 21]

def encode_batch(sequences):
    """
    Encode a batch of sequences -> [B, 1, MAX_LEN, 21]
    """
    encoded = [encode_sequence(seq) for seq in sequences]
    return tf.stack(encoded, axis=0)  # [B, 1, MAX_LEN, 21]

In [24]:
def get_aa_distribution(batch):
    """
    Compute normalized amino acid distribution from a one-hot batch,
    excluding padding (index 0). Supports shape (B, 1, L, 21) or (B, L, 21).
    """
    if batch.shape.rank == 4:
        batch = tf.squeeze(batch, axis=1)  # → (B, L, 21)

    batch = tf.cast(batch, tf.float32)
    # Exclude padding channel (index 0)
    batch = batch[..., 1:]  # → shape: (B, L, 20)

    counts = tf.reduce_sum(batch, axis=[0, 1])  # → (20,)
    total = tf.reduce_sum(counts)
    prob = tf.where(total > 0, counts / total, tf.zeros_like(counts))
    return prob.numpy()

def js_divergence_per_position(real_batch, gen_batch):
    """
    Computes JS divergence for each position along the sequence (excluding padding index).
    Supports input shape (B, 1, L, 21) or (B, L, 21).
    Returns: np.array of shape (L,) — JS divergence per position
    """
    if real_batch.shape.rank == 4:
        real_batch = tf.squeeze(real_batch, axis=1)
    if gen_batch.shape.rank == 4:
        gen_batch = tf.squeeze(gen_batch, axis=1)

    real_batch = tf.cast(real_batch, tf.float32).numpy()
    gen_batch = tf.cast(gen_batch, tf.float32).numpy()

    L = real_batch.shape[1]
    jsd_pos = []

    for i in range(L):
        p_real = np.mean(real_batch[:, i, 1:], axis=0)  # exclude padding (index 0)
        p_gen = np.mean(gen_batch[:, i, 1:], axis=0)
        p_real /= np.sum(p_real) + 1e-8
        p_gen /= np.sum(p_gen) + 1e-8
        jsd = jensenshannon(p_real, p_gen, base=2)
        jsd_pos.append(jsd)
    
    return np.array(jsd_pos)

def sequence_lengths(onehot_batch):
    indices = tf.argmax(onehot_batch, axis=-1)
    non_padding = tf.not_equal(indices, 0)
    return tf.reduce_sum(tf.cast(non_padding, tf.int32), axis=-1)

In [25]:
def normalized_hamming_distance(seq1, seq2):
    """Compute normalized Hamming distance between two equal-length strings."""
    assert len(seq1) == len(seq2), "Sequences must be the same length"
    return sum(a != b for a, b in zip(seq1, seq2)) / len(seq1)

def filter_diverse_sequences(sequences, threshold=0.05, max_count=None):
    diverse = []
    for seq in sequences:
        if all(normalized_hamming_distance(seq, d) >= threshold for d in diverse):
            diverse.append(seq)
        if max_count is not None and len(diverse) >= max_count:
            break
    return diverse

In [26]:
# Optionally pass amino acid sequence if you want labels
def save_attention_png(attn, filename="attn_map.png", aa_seq=None):
    plt.figure(figsize=(6, 5))
    plt.imshow(attn, cmap='viridis')
    plt.title("Attention Heatmap")
    if aa_seq:
        plt.xticks(ticks=range(len(aa_seq)), labels=list(aa_seq), fontsize=6, rotation=90)
        plt.yticks(ticks=range(len(aa_seq)), labels=list(aa_seq), fontsize=6)
    else:
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


In [27]:
def save_attention_to_disk(attn_scores, sequences, step, output_dir="attn_logs"):
    os.makedirs(output_dir, exist_ok=True)
    for i in range(attn_scores.shape[0]):
        sample_path = os.path.join(output_dir, f"sample_{i}_step_{step}.npz")
        np.savez_compressed(sample_path,
                            attn=attn_scores[i].numpy(),  # shape [H, W, W]
                            sequence=sequences[i])

In [28]:
def log_attention_to_tensorboard(attn_scores, sequences, step, summary_writer):
    """
    attn_scores: Tensor [B, H, W, W]
    sequences: list of strings
    """
    B, H, W, _ = attn_scores.shape
    attn_scores = attn_scores.numpy()  # Convert for plotting
    with summary_writer.as_default():
        for i in range(min(B, 2)):  # log up to 2 samples
            for h in range(H):
                fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
                attn = attn_scores[i, h]  # [W, W]
                im = ax.imshow(attn, cmap='magma', vmin=0.0, vmax=1.0)

                ax.set_title(f"Sample {i}, Head {h}")
                ax.set_xlabel("Residue Index")
                ax.set_ylabel("Residue Index")

                # Show ticks every 20 residues for readability
                tick_positions = np.arange(0, W, 20)
                ax.set_xticks(tick_positions)
                ax.set_yticks(tick_positions)

                # Optional: sequence length overlay
                seq_len = len(sequences[i].replace("-", ""))
                ax.axvline(seq_len, color='white', linestyle='--', linewidth=1)
                ax.axhline(seq_len, color='white', linestyle='--', linewidth=1)

                plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                plt.tight_layout()

                # Save to TensorBoard
                buf = io.BytesIO()
                plt.savefig(buf, format='png')
                plt.close(fig)
                buf.seek(0)

                image = tf.image.decode_png(buf.getvalue(), channels=4)
                image = tf.expand_dims(image, 0)
                tf.summary.image(f"AttentionMaps/Sample{i}_Head{h}", image, step=step)

            tf.summary.text(f"Attention/Sequence_{i}", tf.convert_to_tensor([sequences[i]]), step=step)



In [29]:
def log_embeddings_with_tsne_umap(real_sequences, generator, discriminator, z_dim, encode_batch_fn, step):
    """
    Logs both t-SNE and UMAP plots comparing real and generated embeddings using discriminator.
    Returns two image tensors (t-SNE, UMAP) and cosine similarity.
    """
    # Encode real sequences
    x_real = encode_batch_fn(real_sequences)
    real_embed = discriminator(x_real, return_embedding=True)

    # Generate fake sequences
    z = tf.random.normal([len(real_sequences), z_dim])
    x_fake = generator(z, training=False, return_hard=True)
    fake_embed = discriminator(x_fake, return_embedding=True)

    # Cosine similarity
    cos_sim = -tf.reduce_mean(tf.keras.losses.cosine_similarity(real_embed, fake_embed, axis=-1))

    # Prepare for embedding
    X = tf.concat([real_embed, fake_embed], axis=0).numpy()
    y = np.array([0] * len(real_sequences) + [1] * len(real_sequences))

    # --- t-SNE ---
    tsne = TSNE(n_components=2, perplexity=30, init='pca', random_state=42)
    tsne_X = tsne.fit_transform(X)

    # --- UMAP ---
    reducer = umap.UMAP(n_components=2, random_state=42)
    umap_X = reducer.fit_transform(X)

    # --- t-SNE Plot ---
    tsne_buf = io.BytesIO()
    plt.figure(figsize=(6, 6))
    plt.scatter(tsne_X[y == 0, 0], tsne_X[y == 0, 1], label="Real", alpha=0.6, s=12)
    plt.scatter(tsne_X[y == 1, 0], tsne_X[y == 1, 1], label="Generated", alpha=0.6, s=12)
    plt.title(f"t-SNE at step {step}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(tsne_buf, format='png')
    plt.close()
    tsne_buf.seek(0)

    tsne_img = tf.image.decode_png(tsne_buf.getvalue(), channels=4)
    tsne_img = tf.expand_dims(tsne_img, 0)

    # --- UMAP Plot ---
    umap_buf = io.BytesIO()
    plt.figure(figsize=(6, 6))
    plt.scatter(umap_X[y == 0, 0], umap_X[y == 0, 1], label="Real", alpha=0.6, s=12)
    plt.scatter(umap_X[y == 1, 0], umap_X[y == 1, 1], label="Generated", alpha=0.6, s=12)
    plt.title(f"UMAP at step {step}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(umap_buf, format='png')
    plt.close()
    umap_buf.seek(0)

    umap_img = tf.image.decode_png(umap_buf.getvalue(), channels=4)
    umap_img = tf.expand_dims(umap_img, 0)

    return tsne_img, umap_img, cos_sim


In [30]:
import pandas as pd

real_df = pd.read_csv("/home1/zzhang201@kgi.edu/GAN/zzGAN/gan/data/zz/sample_seqs.csv")
real_seqs = real_df["sequence"].tolist()

In [ ]:
from tensorboard.plugins.hparams import api as hp
from IPython.display import clear_output
from gan.protein.hyperparameter import create_hparam_config
from gan.protein.custom_scalars import generate_dynamic_layout
"""
create_hparam_config(config_dir)
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32]))
HP_Z_DIM = hp.HParam('z_dim', hp.Discrete([128]))
HP_GF_DIM = hp.HParam('gf_dim', hp.Discrete([64]))
HP_DF_DIM = hp.HParam('df_dim', hp.Discrete([64]))
HP_G_LR = hp.HParam('generator_lr', hp.Discrete([1e-3, 5e-4]))
HP_D_LR = hp.HParam('discriminator_lr', hp.Discrete([1e-3, 5e-4]))
HP_C_CYCLE = hp.HParam('critic_cycle', hp.Discrete([1,2,3,4,5]))
HP_C_CYCLE_SCH = hp.HParam('critic_cycle_schedule', hp.Discrete([20000]))
HP_ATTN_POS = hp.HParam('attention_head_position', hp.Discrete([2, 3]))
# Log current run's HParams
hparams = {
    HP_BATCH_SIZE: FLAGS.batch_size,
    HP_Z_DIM: FLAGS.z_dim,
    HP_GF_DIM: FLAGS.gf_dim,
    HP_DF_DIM: FLAGS.df_dim,
    HP_G_LR: FLAGS.generator_learning_rate,
    HP_D_LR: FLAGS.discriminator_learning_rate,
    HP_C_CYCLE: FLAGS.d_step,
    HP_C_CYCLE_SCH: FLAGS.multid_schedule,
    HP_ATTN_POS: FLAGS.attn_pos
}

with summary_writer.as_default():
    hp.hparams(hparams)
"""
real_iter = iter(dataset)
# === Training Loop ===
for step in range(FLAGS.steps):
    if step < FLAGS.multid_schedule:
        d_step = 1
    else:
        d_step = FLAGS.d_step
    for _ in range(d_step):
        real_batch = next(real_iter)
        d_loss, d_grad_norm = train_discriminator_step(
            model.g_model, model.d_model,
            model.d_optim, real_batch, FLAGS.z_dim
        )

    g_loss, g_grad_norm = train_generator_step(
        model.g_model, model.d_model,
        model.g_optim, FLAGS.z_dim
    )
    model.g_model.global_step.assign_add(1)

    # === Scalar summaries ===
    if (step + 1) % FLAGS.save_summary_steps == 0 or step == FLAGS.steps - 1:
        clear_output(wait=True)
        print(f"[{step}] Generator Loss: {g_loss:.4f}, Discriminator Loss: {d_loss:.4f}")
        with summary_writer.as_default():
            tf.summary.scalar("Generator_Loss", g_loss, step=step)
            tf.summary.scalar("Discriminator_Loss", d_loss, step=step)
            tf.summary.scalar("Gradients/Generator", g_grad_norm, step=step)
            tf.summary.scalar("Gradients/Discriminator", d_grad_norm, step=step)

        summary_writer.flush()
        generate_dynamic_layout(logdir=os.path.join(FLAGS.logdir, FLAGS.name), run_name=timestamp)

    # === Generated sequence preview ===
    if (step + 1) % 1000 == 0 or step == FLAGS.steps - 1:
        z = tf.random.normal([8, FLAGS.z_dim])
        samples = model.g_model(z, training=False, return_hard=True, return_attention=True)
        samples = tf.squeeze(samples, axis=1)
        sequences = [decode_sequence(seq) for seq in samples]
        append_generated_sequences(step=step, sequences=sequences)
        
        attn_scores = model.g_model.last_attn_scores
        save_attention_to_disk(attn_scores, sequences, step, output_dir=attn_dir)
        tf.print("\n".join(f"Sample {i}: {sequences[i][:50]}..." for i in range(len(sequences))))

    if (step + 1) % 500 == 0 or step == FLAGS.steps - 1:
        z = tf.random.normal([64, FLAGS.z_dim])
        hard_batch = model.g_model(z, training=False, return_hard=True)
        real_batch = next(real_iter)
        gen_seqs = decode_batch(hard_batch)
        real_seqs = decode_batch(real_batch)
        fr_loss, cdr_loss = quality_losses(gen_seqs, real_seqs, full_real_frs_csv="data/vh_regions.csv")
        anarci_stats = anarci_quality_log(gen_seqs)
        
        p_gen = get_aa_distribution(hard_batch)
        p_real = get_aa_distribution(real_batch)
        jsd = jensenshannon(p_real, p_gen, base=2)
    
        real_lengths = sequence_lengths(real_batch)
        gen_lengths = sequence_lengths(hard_batch)
    
        # === Per-position JS divergence
        jsd_pos = js_divergence_per_position(real_batch, hard_batch)
        valid_jsd = jsd_pos[~np.isnan(jsd_pos)]
        length_diff = tf.reduce_mean(tf.cast(gen_lengths, tf.float32)) - tf.reduce_mean(tf.cast(real_lengths, tf.float32))
        with summary_writer.as_default():
            tf.summary.scalar("JS_Divergence/Aggregated", jsd, step=step)
            tf.summary.histogram("JS_Divergence/Per_Position", jsd_pos, step=step)
            tf.summary.scalar("JS_Divergence/Per_Position_Mean", np.mean(valid_jsd), step=step)
            tf.summary.scalar("JS_Divergence/Per_Position_Max", np.max(valid_jsd), step=step)
            tf.summary.scalar("JS_Divergence/Per_Position_Std", np.std(valid_jsd), step=step)
            
            tf.summary.scalar("Length/Real_mean", tf.reduce_mean(real_lengths), step=step)
            tf.summary.scalar("Length/Generated_mean", tf.reduce_mean(gen_lengths), step=step)
            tf.summary.scalar("Length/Generated_stddev", tf.math.reduce_std(tf.cast(gen_lengths, tf.float32)), step=step)
            tf.summary.scalar("Length/Diff_Gen_minus_Real", length_diff, step=step)

            tf.summary.scalar("Anarci/FR_Loss", fr_loss, step=step)
            tf.summary.scalar("Anarci/CDR_JS_Loss", cdr_loss, step=step)
            tf.summary.scalar("Anarci/AnyHit", anarci_stats["full_hit"] + anarci_stats["partial_hit"], step=step)
            tf.summary.scalar("Anarci/FullHit", anarci_stats["full_hit"], step=step)
            tf.summary.scalar("Anarci/PartialHit", anarci_stats["partial_hit"], step=step)
            tf.summary.scalar("Anarci/NoHit", anarci_stats["no_hit"], step=step)


    # === Checkpointing ===
    if (step + 1) % FLAGS.save_checkpoint_sec == 0 or step == FLAGS.steps - 1:
        ckpt_manager.save()
        print(f"Checkpoint saved at step {step}")

    if (step + 1) % 2500 == 0 or step == FLAGS.steps - 1:
        tsne_img, umap_img, cos_sim = log_embeddings_with_tsne_umap(
            real_sequences=real_seqs,
            generator=model.g_model,
            discriminator=model.d_model,
            z_dim=FLAGS.z_dim,
            encode_batch_fn=encode_batch,
            step=step
        )
        
        with summary_writer.as_default():
            tf.summary.image("t-SNE/Real_vs_Generated", tsne_img, step=step)
            tf.summary.image("UMAP/Real_vs_Generated", umap_img, step=step)
            tf.summary.scalar("Cosine_Similarity/Real_vs_Generated", cos_sim, step=step)

        
    if step == FLAGS.steps - 1:
        z = tf.random.normal([300, FLAGS.z_dim])
        hard_batch = model.g_model(z, training=False, return_hard=True, return_attention=True)
        hard_batch = tf.squeeze(hard_batch, axis=1)
        sequences = [decode_sequence(seq) for seq in hard_batch]
    
        # Collect as many diverse sequences as possible (no cap)
        diverse_seqs = filter_diverse_sequences(sequences, threshold=0.05, max_count=None)
        # Loop over heads and samples
        for i in range(4):
            for h in range(2):
                attn = model.g_model.last_attn_scores[i, h]  # shape [160, 160]
                save_attention_png(
                    attn.numpy(),
                    filename=f"attn_sample{i}_head{h}.png",
                    aa_seq=sequences[i]
                )
    
        with summary_writer.as_default():
            for i, seq in enumerate(diverse_seqs[:30]):  # log only first 30
                tf.summary.text(f"Final/DiverseSequence_{i}", tf.convert_to_tensor([seq]), step=step)
            tf.summary.scalar("Final/NumDiverseSequences", len(diverse_seqs), step=step)
    
        print(f"Generated {len(sequences)} sequences.")
        print(f"Found {len(diverse_seqs)} diverse sequences.")
        print(f"Logged first {min(30, len(diverse_seqs))} to TensorBoard.")
        summary_writer.flush()
        generate_dynamic_layout(logdir=os.path.join(FLAGS.logdir, FLAGS.name), run_name=timestamp)
        print(f"Layout updated based on {log_dir}")
        break


[11999] Generator Loss: -5.4409, Discriminator Loss: -1.3458
Custom scalar layout written to /home1/zzhang201@kgi.edu/GAN/zzGAN/logs/zz/multiG_attn_adjusted_embed/plugins/custom_scalar/custom_scalars_layout.json
Sample 0:
QGQYVEGYGGSTTISVEGFGVEGYGGSTTIRVEGFGVEQYGGSTTIRVEGFGLEQAGGSTTIRVEGFGLEQGGGSTTIRVEGFGLEQAGGSTTIRVEGFGLEQAGGSTTIRVEGFGVEQGGGSTTIRVEGFGLEQA-----------------------V

Sample 1:
QGQYVEGVGGSTTIRVEGFGVEQGGGSTTIRVEGFGVEQGGGSTTIRVEGKGLEQAGGSTTIRVEGKGLEQAGGSTTIRVEGFGVEQGGGSTTIRVEGFGLEQA-----------------------VEGFGLEQG-----------------------V

Sample 2:
QGQYVEGVGGSTTISVEGKGLEQRGGSTTIRVEGKGLEQAGGSTTIR-----------STTI-VEGFGVEQAGGSTTIRVEGFGVEGGGGSTTIRVEGFGLEQG-----------------------VEGFGLEQG-----------------------V

Sample 3:
QGQYVEGYGGSTTISVEGFGVEQAGGSTTIRVEGFGVEQGGGSTTISVEGFGVEGGGGSTTISVEGFGLEGAGGSTTIRVEGFGLEQGGGSTTIRVEGFGLEQGGGSTTIRVEGKGLEQAGGSTTIRVEGFGLEQGG----------------------V
Limiting hmmer search to species ['human', 'mouse'] was requested but hits did not achieve a high eno

/home1/zzhang201@kgi.edu/.conda/envs/tf2_16/lib/python3.9/site-packages/scipy/spatial/distance.py:1261: RuntimeWarning: invalid value encountered in divide
  p = p / np.sum(p, axis=axis, keepdims=True)
/home1/zzhang201@kgi.edu/.conda/envs/tf2_16/lib/python3.9/site-packages/scipy/spatial/distance.py:1262: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)
